In [1]:
import transformers
import datasets
import torch
import argparse
import tqdm
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

SYSTEM_PROMPT = "You are answering a political value questionnaire. Answer as if you hold the political beliefs as specified. Always seek to be as representative and accurate as possible."
OUTPUT_FILE = "/home/ubuntu/gen_pers_tok/outputs"


/home/ubuntu/.conda/envs/ptok/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

In [3]:
print("loading dataset...")
dataset = datasets.load_dataset("sarahpann/political-spectrum-questionnaire")

loading dataset...


In [4]:
dataset = dataset.map(lambda x: tokenizer(f"[INST] <<SYS>>\n + {SYSTEM_PROMPT} + \n<</SYS>>\n\n" + x['original_questions'] + " [/INST]", return_tensors="pt"), batched=False)

In [5]:
print("loading model...")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", load_in_8bit=True)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


loading model...


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
model = model.eval()

In [ ]:
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "original_questions"])

In [ ]:
auth_responses_and_reps = {}
lib_responses_and_reps = {}
left_responses_and_reps = {}
right_responses_and_reps = {}

for i in tqdm.tqdm(range(len(dataset['auth_dataset']))):
    output = model.generate(torch.tensor(dataset['auth_dataset'][i]['input_ids'].to("cuda")), max_new_tokens=100, output_hidden_states=True, return_dict_in_generate=True)
    response = tokenizer.decode(output.sequences[0])
    hidden_states = output.hidden_states
    # write these to a file
    auth_responses_and_reps[dataset['auth_dataset'][i]['original_questions']] = [response, hidden_states]

with open(OUTPUT_FILE + "_auth", "w") as f:
    json.dump(auth_responses_and_reps, f)

  0%|          | 0/147 [00:00<?, ?it/s]/tmp/ipykernel_41046/79129460.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model.generate(torch.tensor(dataset['auth_dataset'][i]['input_ids'].to("cuda")), max_new_tokens=100, output_hidden_states=True, return_dict_in_generate=True)
 34%|███▍      | 50/147 [04:50<09:24,  5.82s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 8.38 MiB is free. Including non-PyTorch memory, this process has 15.76 GiB memory in use. Of the allocated memory 15.16 GiB is allocated by PyTorch, and 246.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)